In [2]:
import imp
import json
from pathlib import Path
from tqdm import tqdm

In [4]:
paths = list(Path("./sample-data/").glob("*.json"))
save_dir = Path("resumes")
save_dir.mkdir(exist_ok=True)


In [11]:
from pprint import pprint


pprint(data['education_information'])

{'data': [{'end_date': '01-06-2021',
           'institution': 'Academy of Finance, Hanoi, Vietnam',
           'major_description': "Bachelor's degree, Accounting and Finance",
           'start_date': '01-01-2021',
           'summary': 'Designed and developed a Capsule based Deep Neural '
                      'Network for emotion classification using EEG signals '
                      'collected from the brain.'},
          {'end_date': '01-10-2021',
           'institution': 'Hanoi University',
           'major_description': "Bachelor's degree, Japanese Studies",
           'start_date': '01-07-2021',
           'summary': 'Designed and developed a Capsule based Deep Neural '
                      'Network for emotion classification using EEG signals '
                      'collected from the brain.'}]}


In [9]:
data.keys()

dict_keys(['applied_position', 'basic_information', 'work_experiences', 'education_information', 'projects', 'awards', 'skills'])

In [23]:
def make_profile(url: str):
    if url is None or len(url.strip()) == 0:
        return None
    parts = url.replace("https://www.", "").replace("https://", "").split("/")
    if len(parts) >= 2:
        return {
            "network": parts[0],
            "username": parts[-1],
            "url": url,
        }
    else:
        return None


In [30]:
for path in tqdm(paths):
    with open(path, "r") as f:
        data = json.load(f)

    jsonresume_data = {
        "basics": {
            "name": data["basic_information"]["data"]["name"],
            "label": data["applied_position"],
            "email": data["basic_information"]["data"]["email"],
            "phone": data["basic_information"]["data"]["phone"],
            "summary": data["basic_information"]["data"]["summary"],
            "location": {
                "address": data["basic_information"]["data"]["address"],
                "region": data["basic_information"]["data"]["city"],
            },
            "profiles": [
                profile
                for profile in [
                    make_profile(url) for url in data["basic_information"]['data']["profiles"]
                ]
            ],
        },
        "work": [
            {
                "company": work_experience["company"],
                "position": work_experience["position"],
                "startDate": work_experience["start_date"],
                "endDate": work_experience["end_date"],
                "summary": work_experience["summary"],
            }
            for work_experience in data["work_experiences"]["data"]
        ],
        "education": [
            {
                "institution": item["institution"],
                "area": item["major_description"],
                "startDate": item["start_date"],
                "endDate": item["end_date"],
            }
            for item in data["education_information"]["data"]
        ],
        "awards": [
            {
                "title": item["name"],
                "summary": item["summary"],
            }
            for item in data["awards"]["data"]
        ],
        "skills": [
            {
                "name": skill,
            } for skill in data['skills']['data']
        ],
        "projects": [
            {
                "name": item["name"],
                "description": item["summary"] or item["description"],
            }
            for item in data["projects"]["data"]
        ],
    }
    with open(save_dir / path.name, 'w') as f:
        json.dump(jsonresume_data, f)


100%|██████████| 150/150 [00:00<00:00, 376.95it/s]


: 